In [ ]:
from bloqade import qasm2
import numpy as np
from bloqade.pyqrack import PyQrack
from collections import Counter
from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin import ir
from kirin.rewrite import Walk
from bloqade.qasm2.passes import UOpToParallel, QASM2Fold

import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(r'src\TimeIndepSpinChain')
from TimeIndepSpinChain import (
    SpinChainLieTrotter, SpinChainSuzukiTrotter
)

# O(n^2) Trotterization

In [ ]:
time_interval = 0.5
steps = 24
N = 2
mag_vals = [] 

for i in range(steps):
    kernel_PT = SpinChainLieTrotter(N, time_interval*i, i)
    
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results = device.multi_run(kernel_PT, _shots=100)
    
    def to_bitstrings(results):
        return Counter(map(lambda result:"".join(map(str, result)), results))
    
    counts = to_bitstrings(results)
    
    total_magnetization = 0
    
    for key, value in counts.items():
        spin_down = str(key).count('1')
        total_magnetization += value*spin_down * (-1) + value*(2**N - spin_down)

    print("Time " + str(time_interval*i) + ": " + str(total_magnetization/100))
    mag_vals.append(total_magnetization/100) 

In [ ]:
plt.figure(figsize=(10, 4))
t = np.linspace(0, (steps-1)*time_interval, 48)
magnetization = np.array(mag_vals)
plt.plot(t, magnetization, marker="x", c="darkmagenta", ls="dashed", lw=0.8)
plt.xlabel("Time (a.u.)")
plt.ylabel(r"$\langle M \rangle$")
plt.title("1D Ising chain magnetization over time on Bloqade (4 qubits, no noise, O(t^2))") 

# Better Trotterization

#### Test on 4 Qubits

In [ ]:
time_interval = 0.5
steps = 24
N = 2
mag_vals = [] 

for i in range(steps):
    kernel_PT = SpinChainSuzukiTrotter(N, time_interval*i, i)
    
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results = device.multi_run(kernel_PT, _shots=100)
    
    def to_bitstrings(results):
        return Counter(map(lambda result:"".join(map(str, result)), results))
    
    counts = to_bitstrings(results)
    
    total_magnetization = 0
    
    for key, value in counts.items():
        spin_down = str(key).count('1')
        total_magnetization += value*spin_down * (-1/2) + value*(2**N - spin_down) * (1/2)

    print("Time " + str(time_interval*i) + ": " + str(total_magnetization/100))
    mag_vals.append(total_magnetization/100) 

In [ ]:
plt.figure(figsize=(10, 4))
t = np.linspace(0, (steps-1)*time_interval, 24)
magnetization = np.array(mag_vals)
plt.plot(t, magnetization, marker="x", c="darkmagenta", ls="dashed", lw=0.8)
plt.xlabel("Time (a.u.)")
plt.ylabel(r"$\langle M \rangle$")
plt.title("1D Ising chain magnetization over time on Bloqade (4 qubits, no noise, O(t^3))") 